**Package Installations**

In [ ]:
!pip install tensorflow-gpu==1.15.5
!pip install git+https://github.com/openai/baselines.git@ea25b9e8b234e6ee1bca43083f8f3cf974143998

**Library Imports**

In [ ]:
import time
import gym
import random
import functools
import numpy as np
import os.path as osp

from tensorflow import keras
from tensorflow.keras import layers

from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from baselines import deepq
from baselines import bench
from baselines import logger
import tensorflow as tf
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.cluster import KMeans
from baselines.common.tf_util import make_session

from baselines.common import set_global_seeds, explained_variance
from baselines.common import tf_util
from baselines.common.policies import build_policy
from baselines.common.tf_util import get_session, save_variables, load_variables

from baselines.a2c.utils import Scheduler, find_trainable_variables
from baselines.a2c.runner import Runner
from baselines.ppo2.ppo2 import safemean
from collections import deque
from baselines.acktr import kfac

from tensorflow import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D

**Dataset Preprocessing: No Attack**

In [ ]:
concatenated_df = pd.read_csv('n-baiot_combined.csv')
concatenated_df.columns = concatenated_df.columns.str.strip()
print("original length of df:", len(concatenated_df))
concatenated_df.replace([np.inf, -np.inf], np.nan, inplace=True)
concatenated_df.dropna(inplace=True)
print("after droping null values, the length of df:", len(concatenated_df))
normal_class = 'benign'
output_column = 'Labels_binary'
concatenated_df[output_column] = concatenated_df['type'].apply(lambda x: 0 if x==normal_class else 1)
concatenated_df.head()
X = concatenated_df.drop(columns=[output_column]).copy()
Y = concatenated_df[output_column]
class_le = LabelEncoder()
Y = class_le.fit_transform(Y)
multiclass_le = LabelEncoder()
X['type'] = multiclass_le.fit_transform(X['type'])
remove = []
cat_cols = []
for c in X.columns:
    
# Columns with unique value
    if X[c].nunique() == 1:
        remove.append(c)
        
# Categorical columns
    elif X[c].nunique()<10:
        cat_cols.append(c)
    print(c, X[c].nunique())

cat_cols.append('type')
# Remove columns with unique value
X.drop(columns=remove, inplace=True)
X_discrete = X[cat_cols]
number_of_discrete = len(cat_cols)
X.drop(columns=cat_cols, inplace=True)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
num_of_continuous = len(X_scaled.columns)

# combine categorical and scaled dataframes
X_merge=pd.concat([X_scaled, X_discrete], axis=1, join='inner')

indices=X_scaled.index.difference(X_merge.index)
out = [val for i, val in enumerate(Y) if all(i != indices)]
Y_merge=np.array(out)


seed = 10
test_size = 0.2
X_train, X_test, Y_train, Y_test= train_test_split(X_merge, Y_merge, test_size= test_size, random_state=seed)
test_size = 0.25
X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, test_size= test_size, random_state=seed) 
    
Y_train_multi = X_train['type']
Y_test_multi = X_test['type']
X_train.drop(columns=['type'], inplace=True)
X_val.drop(columns=['type'], inplace=True)
X_test.drop(columns=['type'], inplace=True)

**Dataset Features & Plotting**

In [ ]:
count=concatenated_df['type'].value_counts()
values=[]
for i in range(concatenated_df['type'].nunique()):
    values.append(count[i])

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.bar(concatenated_df['type'].unique(),values)
plt.xticks(rotation=90)

# assign bars to a variable so attributes can be accessed
x=concatenated_df['type'].unique()
bars = plt.bar(x, height=values, width=0.8)

# access bar attributes to place text in appropriate location
for bar in bars:
    y = bar.get_height()
    plt.text(bar.get_x(), y, y)
plt.show()
X_train.head()

**Deep Neural Network Implementation and Evaluation: No Attack**

In [ ]:
# Model / data parameters
num_classes = 1
input_shape = X_train.shape[1]

def DNN_Model(X, Y, batch_size=32, epochs=2):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(num_classes, activation="sigmoid")
        ]
    )

    model.summary()

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    start_time = time.time()
    model.fit(X, Y, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    end_time = time.time()

    score = model.evaluate(X_test, Y_test, verbose=0)
    p_pred = model.predict(X_test)
    p_pred = p_pred.flatten()
    y_pred = np.where(p_pred > 0.5, 1, 0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    print(confusion_matrix(Y_test, y_pred))
    print("Training Time:", end_time - start_time)

DNN_Model(X_train,Y_train)

**Reinforcement Learning Environment Creation**

In [ ]:
X_train_reshape=X_train.values.reshape(X_train.shape[0],X_train.shape[1],1)
X_test_reshape=X_test.values.reshape(X_test.shape[0],X_test.shape[1],1)

class IoTEnv(gym.Env):
    def __init__(self, data_per_episode=1, dataset=(X_train_reshape, Y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(concatenated_df['type'].nunique())
        self.observation_space = gym.spaces.Box(low=0, high=1,
                                                shape=(X_train.shape[1], 1),
                                                dtype=np.float32)

        self.data_per_episode = data_per_episode
        self.step_count = 0

        self.x, self.y = dataset #features, labels
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.data_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs

**Deep Q-Network Implementation**

In [ ]:
def IoT_dqn(X,Y):
    env = IoTEnv(data_per_episode=1, dataset=(X, Y))
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.learn(
        env,
        "mlp",
        num_layers=1,
        num_hidden=64,
        activation=tf.nn.relu,
        hiddens=[32],
        dueling=True,
        lr=1e-3,
        total_timesteps=int(1.0e5),
        buffer_size=10000,
        exploration_fraction=0.001,
        exploration_final_eps=0.0001,
        train_freq=4,
        learning_starts=10000,
        target_network_update_freq=1000,
    )

    env.close()

    return model

start_time = time.time()
dqn_model1 = IoT_dqn(X_train_reshape,Y_train)
print("DQN Training Time:", time.time() - start_time)

**Deep Q-Network Evaluation: No Attack**

In [ ]:
def IoT_dqn_eval(dqn_model):
    attempts, correct, tp, fp, fn, tn, i = 0,0,0,0,0,0,0

    env = IoTEnv(data_per_episode=1, dataset=(X_test_reshape, Y_test), random=False)

    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

                attempts += 1
                if rew > 0:
                  correct += 1
                if Y_test[i]==0 and rew == 1:
                    tp += 1
                elif Y_test[i]==0 and rew == 0:
                    fp += 1
                elif Y_test[i]==1 and rew == 0:
                    fn += 1
                elif Y_test[i]==1 and rew == 1:
                    tn += 1
                i+=1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))
        print('False Positive Rate: {0}%'.format((float(fp) / (fp+tn)) * 100))
        print('Confusion Matrix:')
        print(tp,fp)
        print(fn,tn)
        print('Total no.of test values:',attempts)

IoT_dqn_eval(dqn_model1)

**Proximal Policy Optimization (PPO) Implementation**

In [ ]:
def IoT_ppo(X,Y):
    env = DummyVecEnv([lambda: bench.Monitor(IoTEnv(data_per_episode=1,dataset=(X, Y)), logger.get_dir())])

    model = ppo2.learn(
        env=env,
        network='mlp',
        num_layers=2,
        num_hidden=64,
        nsteps=16,
        total_timesteps=int(1.0e5),
        seed=int(time.time()))

    return model

start_time = time.time()
ppo_model1 = IoT_ppo(X_train_reshape,Y_train)
print("PPO Training Time:", time.time() - start_time)

**Proximal Policy Optimization (PPO) Evaluation: No Attack**

In [ ]:
def IoT_ppo_eval(ppo_model):
    attempts, correct, tp, fp, fn, tn, i = 0,0,0,0,0,0,0

    env = DummyVecEnv([lambda: IoTEnv(data_per_episode=1, dataset=(X_test_reshape, Y_test), random=False)])

    try:
        while True:
            obs, done = env.reset(), [False]
            while not done[0]:
                obs, rew, done, _ = env.step(ppo_model.step(obs[None])[0])

                attempts += 1
                if rew > 0:
                  correct += 1
                if Y_test[i]==0 and rew == 1:
                    tp += 1
                elif Y_test[i]==0 and rew == 0:
                    fp += 1
                elif Y_test[i]==1 and rew == 0:
                    fn += 1
                elif Y_test[i]==1 and rew == 1:
                    tn += 1
                i+=1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))
        print('False Positive Rate: {0}%'.format((float(fp) / (fp+tn)) * 100))
        print('Confusion Matrix:')
        print(tp,fp)
        print(fn,tn)
        print('Total no.of test values:',attempts)

IoT_ppo_eval(ppo_model1)

**Actor-Critic Model Creation**

In [ ]:
class Model_a2c(object):
     def __init__(self, policy, env, nsteps,
                  ent_coef=0.01, vf_coef=0.5, max_grad_norm=0.5, lr=7e-4,
                  alpha=0.99, epsilon=1e-5, total_timesteps=int(80e6), lrschedule='linear'):
       sess = tf_util.get_session()
       nenvs = env.num_envs
       nbatch = nenvs*nsteps
       with tf.variable_scope('a2c_model', reuse=tf.AUTO_REUSE):
          # step_model is used for sampling
          step_model = policy(nenvs, 1, sess)

          # train_model is used to train our network
          train_model = policy(nbatch, nsteps, sess)
          A = tf.placeholder(train_model.action.dtype, train_model.action.shape)
          ADV = tf.placeholder(tf.float32, [nbatch])
          R = tf.placeholder(tf.float32, [nbatch])
          LR = tf.placeholder(tf.float32, [])

          # Calculate the loss
          # Total loss = Policy gradient loss - entropy * entropy coefficient + Value coefficient * value loss

          # Policy loss
          neglogpac = train_model.pd.neglogp(A)
          # L = A(s,a) * -logpi(a|s)
          pg_loss = tf.reduce_mean(ADV * neglogpac)

          # Entropy is used to improve exploration by limiting the premature convergence to suboptimal policy.
          entropy = tf.reduce_mean(train_model.pd.entropy())

          # Value loss
          vf_loss = losses.mean_squared_error(tf.squeeze(train_model.vf), R)

          loss = pg_loss - entropy*ent_coef + vf_loss * vf_coef

          # Update parameters using loss
          # 1. Get the model parameters
          params = find_trainable_variables("a2c_model")

          # 2. Calculate the gradients
          grads = tf.gradients(loss, params)
          if max_grad_norm is not None:
              # Clip the gradients (normalize)
              grads, grad_norm = tf.clip_by_global_norm(grads, max_grad_norm)
          grads = list(zip(grads, params))
          # zip aggregate each gradient with parameters associated
          # For instance zip(ABCD, xyza) => Ax, By, Cz, Da

          # 3. Make op for one policy and value update step of A2C
          trainer = tf.train.RMSPropOptimizer(learning_rate=LR, decay=alpha, epsilon=epsilon)

          _train = trainer.apply_gradients(grads)

          lr = Scheduler(v=lr, nvalues=total_timesteps, schedule=lrschedule)

          def train(obs, states, rewards, masks, actions, values):
              # Here we calculate advantage A(s,a) = R + yV(s') - V(s)
              # rewards = R + yV(s')
              advs = rewards - values
              for step in range(len(obs)):
                  cur_lr = lr.value()

              td_map = {train_model.X:obs, A:actions, ADV:advs, R:rewards, LR:cur_lr}
              if states is not None:
                  td_map[train_model.S] = states
                  td_map[train_model.M] = masks
              policy_loss, value_loss, policy_entropy, _ = sess.run(
                  [pg_loss, vf_loss, entropy, _train],
                  td_map
              )
              return policy_loss, value_loss, policy_entropy


          self.train = train
          self.train_model = train_model
          self.step_model = step_model
          self.step = step_model.step
          self.value = step_model.value
          self.initial_state = step_model.initial_state
          self.save = functools.partial(tf_util.save_variables, sess=sess)
          self.load = functools.partial(tf_util.load_variables, sess=sess)
          tf.global_variables_initializer().run(session=sess)


def learn_a2c(
    network,
    env,
    seed=None,
    nsteps=5,
    total_timesteps=int(80e6),
    vf_coef=0.5,
    ent_coef=0.01,
    max_grad_norm=0.5,
    lr=7e-4,
    lrschedule='linear',
    epsilon=1e-5,
    alpha=0.99,
    gamma=0.99,
    log_interval=100,
    load_path=None,
    **network_kwargs):
  
    set_global_seeds(seed)

    # Get the nb of env
    nenvs = env.num_envs
    policy = build_policy(env, network, **network_kwargs)

    # Instantiate the model object (that creates step_model and train_model)
    model = Model_a2c(policy=policy, env=env, nsteps=nsteps, ent_coef=ent_coef, vf_coef=vf_coef,
        max_grad_norm=max_grad_norm, lr=lr, alpha=alpha, epsilon=epsilon, total_timesteps=total_timesteps, lrschedule=lrschedule)
    if load_path is not None:
        model.load(load_path)

    # Instantiate the runner object
    runner = Runner(env, model, nsteps=nsteps, gamma=gamma)
    epinfobuf = deque(maxlen=100)

    # Calculate the batch_size
    nbatch = nenvs*nsteps

    # Start total timer
    tstart = time.time()

    for update in range(1, total_timesteps//nbatch+1):
        # Get mini batch of experiences
        obs, states, rewards, masks, actions, values, epinfos = runner.run()
        epinfobuf.extend(epinfos)

        policy_loss, value_loss, policy_entropy = model.train(obs, states, rewards, masks, actions, values)
        nseconds = time.time()-tstart

        # Calculate the fps (frame per second)
        fps = int((update*nbatch)/nseconds)
        if update % log_interval == 0 or update == 1:
            # Calculates if value function is a good predicator of the returns (ev > 1)
            # or if it's just worse than predicting nothing (ev =< 0)
            ev = explained_variance(values, rewards)
            logger.record_tabular("nupdates", update)
            logger.record_tabular("total_timesteps", update*nbatch)
            logger.record_tabular("fps", fps)
            logger.record_tabular("policy_entropy", float(policy_entropy))
            logger.record_tabular("value_loss", float(value_loss))
            logger.record_tabular("explained_variance", float(ev))
            logger.record_tabular("eprewmean", safemean([epinfo['r'] for epinfo in epinfobuf]))
            logger.record_tabular("eplenmean", safemean([epinfo['l'] for epinfo in epinfobuf]))
            logger.dump_tabular()
    return model

**Actor-Critic Implementation**

In [ ]:
def IoT_a2c(X,Y):
  env = DummyVecEnv([lambda: bench.Monitor(IoTEnv(data_per_episode=1,dataset=(X, Y)), logger.get_dir())])
  network='mlp'
  model=learn_a2c(
      network,
      env,
      seed=int(time.time()),
      nsteps=5,
      total_timesteps=int(1.0e5),
      vf_coef=0.5,
      ent_coef=0.01,
      max_grad_norm=0.5,
      lr=1e-3,
      lrschedule='linear',
      epsilon=1e-5,
      alpha=0.99,
      gamma=0.99,
      log_interval=100)
  
  return model

start_time = time.time()
a2c_model1=IoT_a2c(X_train_reshape,Y_train)
print("A2C Training Time:", time.time() - start_time)

**Actor-Critic Evaluation: No Attack**

In [ ]:
def IoT_a2c_eval(a2c_model):
    attempts, correct, tp, fp, fn, tn, i = 0,0,0,0,0,0,0

    env = DummyVecEnv([lambda: IoTEnv(data_per_episode=1, dataset=(X_test_reshape, Y_test), random=False)])

    try:
        while True:
            obs, done = env.reset(), [False]
            while not done[0]:
                obs, rew, done, _ = env.step(a2c_model.step(obs[None])[0])

                attempts += 1
                if rew > 0:
                  correct += 1
                if Y_test[i]==0 and rew == 1:
                    tp += 1
                elif Y_test[i]==0 and rew == 0:
                    fp += 1
                elif Y_test[i]==1 and rew == 0:
                    fn += 1
                elif Y_test[i]==1 and rew == 1:
                    tn += 1
                i+=1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))
        print('False Positive Rate: {0}%'.format((float(fp) / (fp+tn)) * 100))
        print('Confusion Matrix:')
        print(tp,fp)
        print(fn,tn)
        print('Total no.of test values:',attempts)

IoT_a2c_eval(a2c_model1)

**Kronecker-Factored Approximation Model Creation**

In [ ]:
class Model_acktr(object):

    def __init__(self, policy, ob_space, ac_space, nenvs,total_timesteps, nprocs=32, nsteps=20,
                 ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.25, max_grad_norm=0.5,
                 kfac_clip=0.001, lrschedule='linear', is_async=True):

        self.sess = sess = get_session()
        nbatch = nenvs * nsteps
        with tf.variable_scope('acktr_model', reuse=tf.AUTO_REUSE):
            self.model = step_model = policy(nenvs, 1, sess=sess)
            self.model2 = train_model = policy(nenvs*nsteps, nsteps, sess=sess)

        A = train_model.pdtype.sample_placeholder([None])
        ADV = tf.placeholder(tf.float32, [nbatch])
        R = tf.placeholder(tf.float32, [nbatch])
        PG_LR = tf.placeholder(tf.float32, [])
        VF_LR = tf.placeholder(tf.float32, [])

        neglogpac = train_model.pd.neglogp(A)
        self.logits = train_model.pi

        ##training loss
        pg_loss = tf.reduce_mean(ADV*neglogpac)
        entropy = tf.reduce_mean(train_model.pd.entropy())
        pg_loss = pg_loss - ent_coef * entropy
        vf_loss = tf.losses.mean_squared_error(tf.squeeze(train_model.vf), R)
        train_loss = pg_loss + vf_coef * vf_loss


        ##Fisher loss construction
        self.pg_fisher = pg_fisher_loss = -tf.reduce_mean(neglogpac)
        sample_net = train_model.vf + tf.random_normal(tf.shape(train_model.vf))
        self.vf_fisher = vf_fisher_loss = - vf_fisher_coef*tf.reduce_mean(tf.pow(train_model.vf - tf.stop_gradient(sample_net), 2))
        self.joint_fisher = joint_fisher_loss = pg_fisher_loss + vf_fisher_loss

        self.params=params = find_trainable_variables("acktr_model")

        self.grads_check = grads = tf.gradients(train_loss,params)

        with tf.device('/gpu:0'):
            self.optim = optim = kfac.KfacOptimizer(learning_rate=PG_LR, clip_kl=kfac_clip,\
                momentum=0.9, kfac_update=1, epsilon=0.01,\
                stats_decay=0.99, is_async=is_async, cold_iter=10, max_grad_norm=max_grad_norm)

            # update_stats_op = optim.compute_and_apply_stats(joint_fisher_loss, var_list=params)
            optim.compute_and_apply_stats(joint_fisher_loss, var_list=params)
            train_op, q_runner = optim.apply_gradients(list(zip(grads,params)))
        self.q_runner = q_runner
        self.lr = Scheduler(v=lr, nvalues=total_timesteps, schedule=lrschedule)

        def train(obs, states, rewards, masks, actions, values):
            advs = rewards - values
            for step in range(len(obs)):
                cur_lr = self.lr.value()

            td_map = {train_model.X:obs, A:actions, ADV:advs, R:rewards, PG_LR:cur_lr, VF_LR:cur_lr}
            if states is not None:
                td_map[train_model.S] = states
                td_map[train_model.M] = masks

            policy_loss, value_loss, policy_entropy, _ = sess.run(
                [pg_loss, vf_loss, entropy, train_op],
                td_map
            )
            return policy_loss, value_loss, policy_entropy


        self.train = train
        self.save = functools.partial(save_variables, sess=sess)
        self.load = functools.partial(load_variables, sess=sess)
        self.train_model = train_model
        self.step_model = step_model
        self.step = step_model.step
        self.value = step_model.value
        self.initial_state = step_model.initial_state
        tf.global_variables_initializer().run(session=sess)

def learn_acktr(network, env, seed, total_timesteps=int(40e6), gamma=0.99, log_interval=100, nprocs=32, nsteps=20,
                 ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.25, max_grad_norm=0.5,
                 kfac_clip=0.001, save_interval=None, lrschedule='linear', load_path=None, is_async=True, **network_kwargs):
    set_global_seeds(seed)


    if network == 'cnn':
        network_kwargs['one_dim_bias'] = True

    policy = build_policy(env, network, **network_kwargs)

    nenvs = env.num_envs
    ob_space = env.observation_space
    ac_space = env.action_space
    make_model = lambda : Model_acktr(policy, ob_space, ac_space, nenvs, total_timesteps, nprocs=nprocs, nsteps
                                =nsteps, ent_coef=ent_coef, vf_coef=vf_coef, vf_fisher_coef=
                                vf_fisher_coef, lr=lr, max_grad_norm=max_grad_norm, kfac_clip=kfac_clip,
                                lrschedule=lrschedule, is_async=is_async)
    if save_interval and logger.get_dir():
        import cloudpickle
        with open(osp.join(logger.get_dir(), 'make_model.pkl'), 'wb') as fh:
            fh.write(cloudpickle.dumps(make_model))
    model = make_model()

    if load_path is not None:
        model.load(load_path)

    runner = Runner(env, model, nsteps=nsteps, gamma=gamma)
    epinfobuf = deque(maxlen=100)
    nbatch = nenvs*nsteps
    tstart = time.time()
    coord = tf.train.Coordinator()
    if is_async:
        enqueue_threads = model.q_runner.create_threads(model.sess, coord=coord, start=True)
    else:
        enqueue_threads = []

    for update in range(1, total_timesteps//nbatch+1):
        obs, states, rewards, masks, actions, values, epinfos = runner.run()
        epinfobuf.extend(epinfos)
        policy_loss, value_loss, policy_entropy = model.train(obs, states, rewards, masks, actions, values)
        model.old_obs = obs
        nseconds = time.time()-tstart
        fps = int((update*nbatch)/nseconds)
        if update % log_interval == 0 or update == 1:
            ev = explained_variance(values, rewards)
            logger.record_tabular("nupdates", update)
            logger.record_tabular("total_timesteps", update*nbatch)
            logger.record_tabular("fps", fps)
            logger.record_tabular("policy_entropy", float(policy_entropy))
            logger.record_tabular("policy_loss", float(policy_loss))
            logger.record_tabular("value_loss", float(value_loss))
            logger.record_tabular("explained_variance", float(ev))
            logger.record_tabular("eprewmean", safemean([epinfo['r'] for epinfo in epinfobuf]))
            logger.record_tabular("eplenmean", safemean([epinfo['l'] for epinfo in epinfobuf]))
            logger.dump_tabular()

        if save_interval and (update % save_interval == 0 or update == 1) and logger.get_dir():
            savepath = osp.join(logger.get_dir(), 'checkpoint%.5i'%update)
            print('Saving to', savepath)
            model.save(savepath)
    coord.request_stop()
    coord.join(enqueue_threads)
    return model

**Kronecker-Factored Approximation Implementation**

In [ ]:
def IoT_acktr(X,Y):
  env = DummyVecEnv([lambda: bench.Monitor(IoTEnv(data_per_episode=1,dataset=(X, Y)), logger.get_dir())])
  network='mlp'
  seed=int(time.time())
  model=learn_acktr(network, env, seed, total_timesteps=int(1.0e5), gamma=0.99, log_interval=100, nprocs=32, nsteps=20,
                 ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.25, max_grad_norm=0.5,
                 kfac_clip=0.001, save_interval=None, lrschedule='linear', load_path=None, is_async=True)
  
  return model

with tf.variable_scope("acktr_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  acktr_model1=IoT_acktr(X_train_reshape,Y_train)
  print("ACKTR Training Time:", time.time() - start_time)

**Kronecker-Factored Approximation Evaluation: No Attack**

In [ ]:
def IoT_acktr_eval(acktr_model):
    attempts, correct, tp, fp, fn, tn, i = 0,0,0,0,0,0,0

    env = DummyVecEnv([lambda: IoTEnv(data_per_episode=1, dataset=(X_test_reshape, Y_test), random=False)])

    try:
        while True:
            obs, done = env.reset(), [False]
            while not done[0]:
                obs, rew, done, _ = env.step(acktr_model.step(obs[None])[0])

                attempts += 1
                if rew > 0:
                  correct += 1
                if Y_test[i]==0 and rew == 1:
                    tp += 1
                elif Y_test[i]==0 and rew == 0:
                    fp += 1
                elif Y_test[i]==1 and rew == 0:
                    fn += 1
                elif Y_test[i]==1 and rew == 1:
                    tn += 1
                i+=1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))
        print('False Positive Rate: {0}%'.format((float(fp) / (fp+tn)) * 100))
        print('Confusion Matrix:')
        print(tp,fp)
        print(fn,tn)
        print('Total no.of test values:',attempts)

IoT_acktr_eval(acktr_model1)

**Label Flipping Attack (LFA)**

In [ ]:
X_train_LFA=X_train.copy()
Y_train_LFA=Y_train[:]

clusterer = KMeans(n_clusters=2, random_state=10)
t0=time.time()
cluster_labels = clusterer.fit_predict(X_train_LFA)
sample_silhouette_values = silhouette_samples(X_train_LFA, cluster_labels)
print("sample_silhouette_values=",sample_silhouette_values)

flipped_Y_train=Y_train_LFA
counter=0
for new_index in range(X_train.shape[0]): 
    if (sample_silhouette_values[new_index]<0.35):                           #and (flipped_Y_train[new_index]==0)
          flipped_Y_train[new_index]=abs(flipped_Y_train[new_index]-1)     #flipped_Y_train[new_index]=1
          counter=counter+1

print("Flipped  counter=", counter)         
t1=time.time()
print("Time for Label Flipping Attack =",t1-t0)

**Deep Neural Network Evaluation: LFA**

In [ ]:
DNN_Model(X_train_LFA,flipped_Y_train)

**Deep Q-Network Evaluation: LFA**

In [ ]:
X_LFA_reshape=X_train_LFA.values.reshape(X_train_LFA.shape[0],X_train_LFA.shape[1],1)

with tf.variable_scope("deepq/eps", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  dqn_model2 = IoT_dqn(X_LFA_reshape,flipped_Y_train)
  print("DQN Training Time:", time.time() - start_time)

IoT_dqn_eval(dqn_model2)

**Proximal Policy Optimization (PPO) Evaluation: LFA**

In [ ]:
with tf.variable_scope("ppo2_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  ppo_model2 = IoT_ppo(X_LFA_reshape,flipped_Y_train)
  print("PPO Training Time:", time.time() - start_time)

IoT_ppo_eval(ppo_model2)

**Actor-Critic Evaluation: LFA**

In [ ]:
start_time = time.time()
a2c_model2=IoT_a2c(X_LFA_reshape,flipped_Y_train)
print("A2C Training Time:", time.time() - start_time)

IoT_a2c_eval(a2c_model2)

**Kronecker-Factored Approximation Evaluation: LFA**

In [ ]:
with tf.variable_scope("acktr_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  acktr_model2=IoT_acktr(X_train_reshape,Y_train)
  print("ACKTR Training Time:", time.time() - start_time)

IoT_acktr_eval(acktr_model2)

**Label Based Semi-supervised Defense (LSD)**

In [ ]:
X_train_LSD=X_train.copy()
Y_train_LSD=flipped_Y_train[:]

X_val_LSD=X_val.copy()
Y_val_LSD=Y_val[:]

t4=time.time()

labels = np.full(X_train.shape[0], -1)
for i in range(X_val.shape[0]):
    labels[i] = Y_val_LSD[i]

label_spread = LabelSpreading(kernel='knn', alpha=0.85)
label_propa=LabelPropagation(kernel='knn', gamma=20, n_neighbors=7, max_iter=1000, tol=0.001, n_jobs=None)
label_spread.fit(X_train_LSD, labels)
label_propa.fit(X_train_LSD, labels)
output_labels_spread = label_spread.transduction_
output_labels_propa = label_propa.transduction_

**Deep Neural Network (DNN) for LSD**

In [ ]:
DNN_model=keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Flatten(),
            layers.Dense(16, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(num_classes, activation="sigmoid")
        ]
    )

DNN_model.summary()

DNN_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

DNN_model.fit(X_train_LSD, Y_train_LSD, batch_size=32, epochs=2, validation_split=0.1)

Y_predict_DNN_for_LSD=DNN_model.predict(X_train_LSD, verbose=0)

Y_predict_DNN_LSD_Final=[0]*len(Y_predict_DNN_for_LSD)
for i in range(len(Y_predict_DNN_for_LSD)):
    if Y_predict_DNN_for_LSD[i]<0.5:
          Y_predict_DNN_LSD_Final[i]=0
    else:
            Y_predict_DNN_LSD_Final[i]=1

print(Y_predict_DNN_LSD_Final)

#*******************************************Voting Between DNN , label Propagation and Label Spreading**************************     
Y_predict_LSD_Final=[0]*len(Y_train)
for i in range(len(Y_train)):
    c=Y_train_LSD[i]+Y_predict_DNN_LSD_Final[i]+output_labels_propa[i]+output_labels_spread[i]
    if 3<=c:
        Y_predict_LSD_Final[i]=1
    else:
        Y_predict_LSD_Final[i]=0
t5=time.time()
print("Time for Label Based Semi-supervised Defense =",t5-t4)

**Deep Neural Network Evaluation: LSD**

In [ ]:
DNN_Model(X_train_LSD, np.array(Y_predict_LSD_Final))

**Deep Q-Network Evaluation: LSD**

In [ ]:
X_LSD_reshape=X_train_LSD.values.reshape(X_train_LSD.shape[0],X_train_LSD.shape[1],1)

with tf.variable_scope("deepq/eps", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  dqn_model3 = IoT_dqn(X_LSD_reshape,Y_predict_LSD_Final)
  print("DQN Training Time:", time.time() - start_time)

IoT_dqn_eval(dqn_model3)

**Proximal Policy Optimization (PPO) Evaluation: LSD**

In [ ]:
with tf.variable_scope("ppo2_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  ppo_model3 = IoT_ppo(X_LSD_reshape,Y_predict_LSD_Final)
  print("PPO Training Time:", time.time() - start_time)

IoT_ppo_eval(ppo_model3)

**Actor-Critic Evaluation: LSD**

In [ ]:
start_time = time.time()
a2c_model3=IoT_a2c(X_LSD_reshape,Y_predict_LSD_Final)
print("A2C Training Time:", time.time() - start_time)

IoT_a2c_eval(a2c_model3)

**Kronecker-Factored Approximation Evaluation: LSD**

In [ ]:
with tf.variable_scope("acktr_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  acktr_model3=IoT_acktr(X_LSD_reshape,Y_predict_LSD_Final)
  print("ACKTR Training Time:", time.time() - start_time)

IoT_acktr_eval(acktr_model3)

**Clustering Based Semi-supervised Defense (CSD)**

In [ ]:
X_train_CSD=X_train.copy()
Y_train_CSD=flipped_Y_train[:]

X_val_CSD=X_val.copy()
Y_val_CSD=Y_val[:]

t6=time.time()

Y_predict_val_from_DNN_model=DNN_model.predict(X_val_CSD, verbose=0)

Y_predict_val_from_DNN_model_Final=[0]*len(Y_predict_val_from_DNN_model)
for i in range(len(Y_predict_val_from_DNN_model)):
    if Y_predict_val_from_DNN_model[i]<0.5:
          Y_predict_val_from_DNN_model[i]=0
    else:
          Y_predict_val_from_DNN_model[i]=1
for i in range(len(Y_predict_val_from_DNN_model)):
    Y_predict_val_from_DNN_model_Final[i]= int(Y_predict_val_from_DNN_model[i])
    
adjusted_rand_score_val=metrics.adjusted_rand_score(Y_val_CSD, Y_predict_val_from_DNN_model_Final)
adjusted_mutual_info_score_val=metrics.adjusted_mutual_info_score(Y_val_CSD, Y_predict_val_from_DNN_model_Final) 
homogeneity_score_val=metrics.homogeneity_score(Y_val_CSD, Y_predict_val_from_DNN_model_Final) 
fowlkes_mallows_score_val=metrics.fowlkes_mallows_score(Y_val_CSD, Y_predict_val_from_DNN_model_Final) 

for i in range(20):        #row_train
    Y_temp=Y_val_CSD.copy()

    X_temp = X_val_CSD
    np.append(Y_temp,Y_train_CSD[i])
    
    Y_predict_CNN_compute_CSD=DNN_model.predict(X_temp, verbose=0)
    
    Y_predict_temp=[0]*len(Y_predict_DNN_compute_CSD)
    
    for n in range(len(Y_predict_DNN_compute_CSD)):
        if Y_predict_DNN_compute_CSD[n]<0.5:
              Y_predict_DNN_compute_CSD[n]=0
        else:
                Y_predict_DNN_compute_CSD[n]=1
      
    for m in range(len(Y_predict_DNN_compute_CSD)):
        Y_predict_temp[m]= int(Y_predict_DNN_compute_CSD[m])

    adjusted_rand_score_temp=metrics.adjusted_rand_score(Y_temp, Y_predict_temp)
    adjusted_mutual_info_score_temp=metrics.adjusted_mutual_info_score(Y_temp, Y_predict_temp) 
    homogeneity_score_temp=metrics.homogeneity_score(Y_temp, Y_predict_temp) 
    fowlkes_mallows_score_temp=metrics.fowlkes_mallows_score(Y_temp, Y_predict_temp)
    
    landa1=abs(adjusted_rand_score_temp-adjusted_rand_score_val)
    landa2=abs(adjusted_mutual_info_score_temp-adjusted_mutual_info_score_val)
    landa3=abs(homogeneity_score_temp-homogeneity_score_val)
    landa4=abs(fowlkes_mallows_score_temp-fowlkes_mallows_score_val)
    
    sum_of_diffrences=landa1+landa2+landa3+landa4
    
    if sum_of_diffrences<0.1:
        X_val_CSD = X_val_CSD
        np.append(Y_val_CSD,Y_train_CSD[i])          
        Y_predict_DNN_inside_CSD=DNN_model.predict(X_val_CSD, verbose=0)
        
        Y_predict_DNN_inside_CSD_Final=[0]*len(Y_predict_DNN_inside_CSD)                   #Y_predict_CNN_inside
        for j in range(len(Y_predict_DNN_inside_CSD)):                                     #Y_predict_CNN_inside
            if Y_predict_DNN_inside_CSD[j]<0.5:
                  Y_predict_DNN_inside_CSD[j]=0
            else:
                  Y_predict_DNN_inside_CSD[j]=1
                    
        for k in range(len(Y_predict_DNN_inside_CSD)):                              #Y_predict_CNN_inside
            Y_predict_DNN_inside_CSD_Final[k]= int(Y_predict_DNN_inside_CSD[k])

        adjusted_rand_score_val=metrics.adjusted_rand_score(Y_val_CSD, Y_predict_DNN_inside_CSD_Final)
        adjusted_mutual_info_score_val=metrics.adjusted_mutual_info_score(Y_val_CSD, Y_predict_DNN_inside_CSD_Final) 
        homogeneity_score_val=metrics.homogeneity_score(Y_val_CSD, Y_predict_DNN_inside_CSD_Final) 
        fowlkes_mallows_score_val=metrics.fowlkes_mallows_score(Y_val_CSD, Y_predict_DNN_inside_CSD_Final) 
t7=time.time()
print("Time for Clustering Based Semi-supervised Defense =",t7-t6)

X_train_Final_CSD= X_val_CSD.copy()  
Y_train_Final_CSD=Y_val_CSD.copy()

**Deep Neural Network Evaluation: CSD**

In [ ]:
DNN_Model(X_train_Final_CSD, Y_train_Final_CSD)

**Deep Q-Network Evaluation: CSD**

In [ ]:
X_CSD_reshape=X_train_Final_CSD.values.reshape(X_train_Final_CSD.shape[0],X_train_Final_CSD.shape[1],1)

with tf.variable_scope("deepq/eps", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  dqn_model4 = IoT_dqn(X_CSD_reshape, Y_train_Final_CSD)
  print("DQN Training Time:", time.time() - start_time)

IoT_dqn_eval(dqn_model4)

**Proximal Policy Optimization (PPO) Evaluation: CSD**

In [ ]:
with tf.variable_scope("ppo2_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  ppo_model4 = IoT_ppo(X_CSD_reshape, Y_train_Final_CSD)
  print("PPO Training Time:", time.time() - start_time)

IoT_ppo_eval(ppo_model4)

**Actor-Critic Evaluation: CSD**

In [ ]:
start_time = time.time()
a2c_model4=IoT_a2c(X_CSD_reshape, Y_train_Final_CSD)
print("A2C Training Time:", time.time() - start_time)

IoT_a2c_eval(a2c_model4)

**Kronecker-Factored Approximation Evaluation: CSD**

In [ ]:
with tf.variable_scope("acktr_model", reuse=tf.AUTO_REUSE):
  start_time = time.time()
  acktr_model4=IoT_acktr(X_CSD_reshape, Y_train_Final_CSD)
  print("ACKTR Training Time:", time.time() - start_time)

IoT_acktr_eval(acktr_model4)